### Imports

In [ ]:
import numpy as np 
import pandas as pd
import cv2

import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,InputLayer
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras.layers as L

### Reading Data

In [ ]:
def read_images(path):
    images = []
    labels = []
    dirs = ['notumor','glioma','meningioma','pituitary']
    for i,dir_ in enumerate(dirs):
        path_ = f"{path}/{dir_}"
        for k in os.listdir(path_):
            images.append(f"{path_}/{k}")
            labels.append(dir_)
    return pd.concat([pd.Series(images,name='Image'),pd.Series(labels,name='Class')],axis=1)

In [ ]:
ts_df = read_images(r"/kaggle/input/brain-tumor-mri-dataset/Training")
test_df = read_images(r"/kaggle/input/brain-tumor-mri-dataset/Testing")

In [ ]:
train_df, valid_df = train_test_split(ts_df,  train_size= 0.8, shuffle= True, random_state= 42)

### EDA

In [ ]:
pd.DataFrame(ts_df,columns=["Class"])["Class"].value_counts().plot(kind='pie',autopct='%1.0f%%')
plt.xlabel("Class")
plt.xticks(rotation=0)
plt.ylabel("Count")
plt.title("Class Distribution of Train Images")
plt.show()

In [ ]:
pd.DataFrame(test_df,columns=["Class"])["Class"].value_counts().plot(kind='pie',autopct='%1.0f%%')
plt.xlabel("Class")
plt.xticks(rotation=0)
plt.ylabel("Count")
plt.title("Class Distribution of Test Images")
plt.show()

### Data Preparation

In [ ]:
img_size =  (256,256)
batch_size = 32

In [ ]:
tr_gen = ImageDataGenerator(rescale=1/255,)
ts_gen = ImageDataGenerator(rescale=1/255)

train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'Image', y_col= 'Class', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'Image', y_col= 'Class', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'Image', y_col= 'Class', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= batch_size)

In [ ]:
g_dict = train_gen.class_indices      
classes = list(g_dict.keys())       
images, labels = next(train_gen)    

plt.figure(figsize= (20, 20))

for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(images[i])
    index = np.argmax(labels[i])  
    class_name = classes[index]  
    plt.title(class_name, color= 'black', fontsize= 24)
    plt.axis('off')
plt.tight_layout()
plt.show()

### ANN

In [ ]:
model = Sequential()
model.add(InputLayer(shape=(256,256,3)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(4,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_gen,validation_data= valid_gen, shuffle= False,epochs=10)

In [ ]:
y_pred = model.predict(test_gen)
y_pred = np.argmax(y_pred,axis=1)
y_true = test_gen.classes

In [ ]:
print(classification_report(y_true,y_pred,zero_division=0,target_names=classes))

In [ ]:
cm = confusion_matrix(y_true,y_pred)
cmd = ConfusionMatrixDisplay(cm,display_labels=classes)

cmd.plot()
plt.show()

### CNN

In [ ]:
model = Sequential([
    InputLayer(shape=(256,256,3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_gen,validation_data= valid_gen, shuffle= False,epochs=10)

In [ ]:
y_pred = model.predict(test_gen)
y_pred = np.argmax(y_pred,axis=1)
y_true = test_gen.classes

In [ ]:
print(classification_report(y_true,y_pred,zero_division=0,target_names=classes))

In [ ]:
cm = confusion_matrix(y_true,y_pred)
cmd = ConfusionMatrixDisplay(cm,display_labels=classes)

cmd.plot()
plt.show()

### Transfer Learning using VGG19 Imagenet

In [ ]:
model_vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(256,256,3))

In [ ]:
model_vgg19.summary()

In [ ]:
model = Sequential()
for layer in model_vgg19.layers:
    model.add(layer)

In [ ]:
for i in model.layers:
   i.trainable = False

In [ ]:
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_gen,validation_data= valid_gen, shuffle= False,epochs=10)

In [ ]:
y_pred = model.predict(test_gen)
y_pred = np.argmax(y_pred,axis=1)
y_true = test_gen.classes

In [ ]:
print(classification_report(y_true,y_pred,zero_division=0,target_names=classes))

In [ ]:
cm = confusion_matrix(y_true,y_pred)
cmd = ConfusionMatrixDisplay(cm,display_labels=classes)

cmd.plot()
plt.show()

### Vision Transformer

In [ ]:
num_epochs = 5
n_classes=4
image_size = 256
patch_size = 8  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 2
mlp_head_units = [56, 28]  # Size of the dense layers of the final classifier

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = L.Dense(units, activation = tf.nn.gelu)(x)
        x = L.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(L.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = 'VALID',
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(L.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = L.Dense(units = projection_dim)
        self.position_embedding = L.Embedding(
            input_dim = num_patches, output_dim = projection_dim
        )

    def call(self, patch):
        positions = tf.range(start = 0, limit = self.num_patches, delta = 1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def vision_transformer():
    inputs = L.Input(shape = (image_size, image_size, 3))
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    for _ in range(transformer_layers):
        x1 = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
        attention_output = L.MultiHeadAttention(
            num_heads = num_heads, key_dim = projection_dim, dropout = 0.1
        )(x1, x1)
        x2 = L.Add()([attention_output, encoded_patches])
        x3 = L.LayerNormalization(epsilon = 1e-6)(x2) 
        x3 = mlp(x3, hidden_units = transformer_units, dropout_rate = 0.1)
        encoded_patches = L.Add()([x3, x2])

    representation = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
    representation = L.Flatten()(representation)
    representation = L.Dropout(0.5)(representation)
    features = mlp(representation, hidden_units = mlp_head_units, dropout_rate = 0.5)
    logits = L.Dense(n_classes)(features)
    model = tf.keras.Model(inputs = inputs, outputs = logits)
    
    return model

In [ ]:
model = vision_transformer()    
model.compile(optimizer = 'adam', 
              loss ='categorical_crossentropy',
              metrics = ['accuracy'])


model.fit(train_gen,validation_data= valid_gen, shuffle= False,epochs=10)

In [ ]:
y_pred = model.predict(test_gen)
y_pred = np.argmax(y_pred,axis=1)
y_true = test_gen.classes

In [ ]:
print(classification_report(y_true,y_pred,zero_division=0,target_names=labels))

In [ ]:
cm = confusion_matrix(y_true,y_pred)
cmd = ConfusionMatrixDisplay(cm,display_labels=labels)

cmd.plot()
plt.show()
